# MAJIS ITL reader and exporter

In [ ]:
from majis import read_itl, save_itl

Juice instrument timeline files (ITL) come with two flavors, one in plain text file (a.k.a. EPS) and one in JSON format.
Both files should contains the same logic, a description on of the observation blocks, however they are not strictly the same.

In this package we try to simplify as much as possible the reader and writer of these files.

## Load absolute time ITL as [`EventsDict`](https://docs.planetary-coverage.org/en/stable/api/events.html#planetary_coverage.events.EventsDict)

Load of an ITL file (either EPS or JSON) will regenerate a `EventsDict` object that will group the observations based on their name:

In [2]:
itl_abs = read_itl('ITL_absolute_time.json')

itl_abs

,event,#,t_start,t_stop
0,MAJ_JUP_DISK_SCAN,2,2032-09-23,2032-09-23


In [3]:
itl_abs['MAJ_JUP_DISK_SCAN']

<EventsList> MAJ_JUP_DISK_SCAN (2032-09-23 -> 2032-09-23 | 2 events)

It is also possible to load the same file in text/EPS format:

In [4]:
itl_abs_eps = read_itl('absolute_time.itl')

itl_abs_eps

,event,#,t_start,t_stop
0,MAJ_JUP_DISK_SCAN,2,2032-09-23,2032-09-23


In [5]:
itl_abs_eps['MAJ_JUP_DISK_SCAN']

<EventsList> MAJ_JUP_DISK_SCAN (2032-09-23 -> 2032-09-23 | 2 events)

## Load relative time ITL as [`EventsList`](https://docs.planetary-coverage.org/en/stable/api/events.html#planetary_coverage.events.EventsList) (as a flat list)

```{note}
At the moment, only EPS file supports relative time input.
```

In [6]:
itl_rel = read_itl('relative_time.itl', refs='events.evf', flat=True)

itl_rel

<EventsList> MAJ_JUP_DISK_SCAN (2032-09-23 -> 2032-09-23 | 2 events)

## Concatenate ITL files into a single file

In [7]:
save_itl('output.itl', itl_abs, itl_rel)

PosixPath('output.itl')

In [8]:
!cat output.itl

# MAJIS - SCENARIO=S007_01 OBS_NAME=MAJ_JUP_DISK_SCAN_101 TARGET=JUPITER CU_TREP=100ms
# MAJIS - CU_FRAME=109 BINNING=1 PPE=64 START_ROW_VIS=436 START_ANGLE=-0.27937
# MAJIS - STOP_ANGLE=+0.23385 SYNCHRONOUS=+3 START_SCAN_SPEED=+0.0022421078
# MAJIS - STOP_SCAN_SPEED=+0.0022421078
2032-09-23T02:58:11.000Z  MAJIS  OBS_START  MAJ_JUP_DISK_SCAN
2032-09-23T03:01:59.900Z  MAJIS  OBS_END    MAJ_JUP_DISK_SCAN

# MAJIS - SCENARIO=S007_01 OBS_NAME=MAJ_JUP_DISK_SCAN_102 TARGET=JUPITER CU_TREP=200ms
# MAJIS - CU_FRAME=109 BINNING=4 PPE=64 START_ROW_VIS=802 START_ANGLE=-0.37981
# MAJIS - STOP_ANGLE=+0.13341 SYNCHRONOUS=+3 START_SCAN_SPEED=+0.0022421078
# MAJIS - STOP_SCAN_SPEED=+0.0022421078
2032-09-23T03:04:34.000Z  MAJIS  OBS_START  MAJ_JUP_DISK_SCAN
2032-09-23T03:08:22.900Z  MAJIS  OBS_END    MAJ_JUP_DISK_SCAN

# MAJIS - OBS_KEY=MAJ_JUP_DISK_SCAN OBS_NAME=MAJ_JUP_DISK_SCAN_001 TYPE=OBSERVATION
# MAJIS - OBSERVATION_TYPE=PRIME TARGET=JUPITER POINTING=MAJIS POINTING_DESIGNER=True
# MAJIS - SCENAR

```{note}
- When multiple ITL are provided, they will be concatenate and ordered by **increasing time**.
- Absolute and relative ITL are compatible.
- Observation block must not overlap (can be by-pass with `overlap=True`)
- If no relative reference is provided, the output will be in absolute time (see below for relative time).
```

##  Export ITL in JSON format

In [9]:
save_itl('ITL_output.json', itl_abs)

PosixPath('ITL_output.json')

In [10]:
!cat ITL_output.json

{
  "header": {
    "filename": "ITL_output.json",
    "creation_date": "2026-01-09T10:29:57.731Z",
    "author": "Majis Operations Toolbox"
  },
  "timeline": [
    {
      "name": "MAJ_JUP_DISK_SCAN",
      "unique_id": "MAJ_JUP_DISK_SCAN_001",
      "instrument": "MAJIS",
      "type": "OBSERVATION",
      "observation_type": "PRIME",
      "target": "JUPITER",
      "pointing": "MAJIS",
      "pointing_designer": true,
      "start_time": "2032-09-23T05:15:45.000Z",
      "end_time": "2032-09-23T05:26:15.000Z",
      "parameters": {
        "scenario_id": "S007_01",
        "cu_trep_ms": 500,
        "nb_cu_frames_tot": 300,
        "spatial_binning": 1,
        "ppe": 400,
        "start_row_vi": 100,
        "mirror_flag": "ENABLE",
        "start_angle": -1.31051,
        "stop_angle": 0.10202,
        "scanner_step_per_frame": 0,
        "start_scan_speed": 0.0022421078,
        "stop_scan_speed": 0.0022421078
      },
      "comment": "This comment will be included in the expo

In the case of JSON export, the content of the ITL is checked against the [SOC and MAJIS ITL schemas](https://juicesoc.esac.esa.int/data/schemas/). For example, here, the `itl_rel` is missing the `MIRROR_FLAG` required parameter to be exported as a JSON ITL:

In [11]:
save_itl('ITL_output.json', itl_rel)

ValidationError: 'mirror_flag' is a required property

Failed validating 'required' in schema['properties']['timeline']['items']['properties']['parameters']:
    {'type': 'object',
     'required': ['scenario_id',
                  'cu_trep_ms',
                  'nb_cu_frames_tot',
                  'spatial_binning',
                  'ppe',
                  'start_row_vi',
                  'mirror_flag',
                  'start_angle',
                  'stop_angle',
                  'scanner_step_per_frame',
                  'start_scan_speed',
                  'stop_scan_speed'],
     'properties': {'mission_phase': {'type': 'string',
                                      'description': 'JUICE mission phase '
                                                     'name (TODO: ask SOC '
                                                     'to provide '
                                                     'allowed/standard '
                                                     '

## Export ITL with repesct to a relative time reference:

In [12]:
save_itl(
    'output.itl', itl_abs, itl_rel, ref='23-SEP-2032_00:00:02 EVENT_NAME (COUNT = 1)'
)

PosixPath('output.itl')

In [13]:
!cat output.itl

# Relative time reference:
# 2032-09-23T00:00:02  EVENT_NAME (COUNT = 1)

# MAJIS - SCENARIO=S007_01 OBS_NAME=MAJ_JUP_DISK_SCAN_101 TARGET=JUPITER CU_TREP=100ms
# MAJIS - CU_FRAME=109 BINNING=1 PPE=64 START_ROW_VIS=436 START_ANGLE=-0.27937
# MAJIS - STOP_ANGLE=+0.23385 SYNCHRONOUS=+3 START_SCAN_SPEED=+0.0022421078
# MAJIS - STOP_SCAN_SPEED=+0.0022421078
EVENT_NAME (COUNT = 1)  +000.02:58:09.000  MAJIS  OBS_START  MAJ_JUP_DISK_SCAN
EVENT_NAME (COUNT = 1)  +000.03:01:57.900  MAJIS  OBS_END    MAJ_JUP_DISK_SCAN

# MAJIS - SCENARIO=S007_01 OBS_NAME=MAJ_JUP_DISK_SCAN_102 TARGET=JUPITER CU_TREP=200ms
# MAJIS - CU_FRAME=109 BINNING=4 PPE=64 START_ROW_VIS=802 START_ANGLE=-0.37981
# MAJIS - STOP_ANGLE=+0.13341 SYNCHRONOUS=+3 START_SCAN_SPEED=+0.0022421078
# MAJIS - STOP_SCAN_SPEED=+0.0022421078
EVENT_NAME (COUNT = 1)  +000.03:04:32.000  MAJIS  OBS_START  MAJ_JUP_DISK_SCAN
EVENT_NAME (COUNT = 1)  +000.03:08:20.900  MAJIS  OBS_END    MAJ_JUP_DISK_SCAN

# MAJIS - OBS_KEY=MAJ_JUP_DISK_SCAN OBS_NAME

## Export ITL to CSV

In [14]:
save_itl('output.csv', itl_abs)

PosixPath('output.csv')

In [15]:
!cat output.csv

#OBS_NAME;OBS_START;OBS_END;OBS_KEY;INSTRUMENT;TYPE;OBSERVATION_TYPE;TARGET;POINTING;POINTING_DESIGNER;SCENARIO;CU_TREP;CU_FRAME;BINNING;PPE;START_ROW_VIS;MIRROR_FLAG;START_ANGLE;STOP_ANGLE;SYNCHRONOUS;START_SCAN_SPEED;STOP_SCAN_SPEED;PRIME;ITL;COMMENTS
MAJ_JUP_DISK_SCAN_001;2032-09-23T05:15:45.000Z;2032-09-23T05:26:15.000Z;MAJ_JUP_DISK_SCAN;MAJIS;OBSERVATION;PRIME;JUPITER;MAJIS;True;S007_01;500;300;1;400;100;ENABLE;-1.31051;0.10202;0;0.0022421078;0.0022421078;True;ITL_absolute_time.json;"This comment will be included in the exported ITL file."
MAJ_JUP_DISK_SCAN_002;2032-09-23T06:09:45.000Z;2032-09-23T06:20:15.000Z;MAJ_JUP_DISK_SCAN;MAJIS;OBSERVATION;RIDER;JUPITER;None;False;S007_01;2100;300;2;400;372;ENABLE;1.32935;-0.08318;3;-0.0022421078;-0.0022421078;False;ITL_absolute_time.json;""

## Export ITL to XLSM timeline

In [16]:
save_itl('output.xlsm', itl_abs, ref='23-SEP-2032_00:00:02 CA_REF (COUNT = 1)')

PosixPath('output.xlsm')

In [17]:
from majis import Timeline

Timeline(timeline='output.xlsm')

Timeline('output.xlsm')